In [1]:
import os
os.chdir('../src/models')

In [22]:
### Standard Libs ###

from collections import defaultdict
from numpy.random import default_rng
import numpy as np
import networkx as nx
from itertools import chain

In [23]:
### External Definitions ###

from zone_references import initial_districts
from disease_states import states_dict
from patient_evolution import susceptible_to_exposed, change_state
from actions import city_restrictions

In [24]:
def make_adjacency_list(G):
    adj_list = {}

    for node, neighbors in G.adjacency():
        adj_list[node] = defaultdict(list)
        for neighbor, relations in neighbors.items():
            for i, relation_dict in relations.items():
                relation = relation_dict['edge_type']
                adj_list[node][relation].append(neighbor)
    return adj_list

In [25]:

def make_graph_structures(gpickle_path):
    G = nx.read_gpickle(gpickle_path)
    adj_list = make_adjacency_list(G)
    
    return G, adj_list

In [26]:
def init_infection(gpickle_path, pct=.0005, return_contacts_infected = False):
    """
    Given a Graph G, infects pct% of population and set the
    remainder as susceptible. This is considered day 0.
    Args:
        pct (float): percentage of people initially infected.
    Returns:
        new_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.
    """

    G, adj_list = make_graph_structures(gpickle_path)

    sample_size = int(np.ceil(len(G.nodes()) * pct/len(initial_districts)))
    size = max(sample_size, 1)

    infected = []
    for zones in initial_districts.values():
        init_nodes = ([x for x, v in G.nodes(data=True) 
                           if v['home'] in zones])

        infected.extend(list(np.random.choice(init_nodes, size=size, replace=False)))

    pop_matrix = np.array([[node, states_dict['susceptible'],
                            -1, -1, data['age']]
                          for node, data in G.nodes(data=True)]).astype(int)
    
    contacts_infected = defaultdict(int)

    matrix_change = pop_matrix[np.isin(pop_matrix[:, 0], infected)]

    matrix_keep = pop_matrix[~np.isin(pop_matrix[:, 0], infected)]

    matrix_change = np.apply_along_axis(susceptible_to_exposed,
                                        1, matrix_change, day=0)

    new_matrix = np.concatenate((matrix_keep, matrix_change))
    assert new_matrix.shape == pop_matrix.shape

    if return_contacts_infected:
        return new_matrix, adj_list, contacts_infected

    else:
        return new_matrix, adj_list

In [27]:
def expose_population(pop_matrix, exposed, day):
    """
    Receives the population matrix, an array containing ids of newly
    exposed individuals and the current simulation day
    Args:
        pop_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.
        exposed (list): list of newly exposed people id
        day (int): current simulation day
    Returns:
        new_matrix (np.array): The population matrix with the
        newly exposed people exposed.
    Raises:
        ValueError: If shape of starting matrix is different from final matrix
    """
    matrix_change = pop_matrix[np.isin(pop_matrix[:, 0], exposed)]
    matrix_keep = pop_matrix[~np.isin(pop_matrix[:, 0], exposed)]
    matrix_change = np.apply_along_axis(susceptible_to_exposed,
                                        1, matrix_change, day=day)

    new_matrix = np.concatenate((matrix_keep, matrix_change))

    if new_matrix.shape != pop_matrix.shape:
        raise ValueError("Input and output matrix shapes are different")
    return new_matrix

In [28]:
def lambda_leak_expose(pop_matrix, day, lambda_leak=0.00005):
    """
    Receives the population matrix, the current day and the leak factor.
    Chooses at random a lambda_leak percentage of the population to expose
    Args:
        pop_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.
        lambda_leak (float): the percentage of the population to expose
        day: the current day of simulation
    Returns:
        new_matrix (np.array): The population matrix with the
                               newly exposed people.
    Raises:
        ValueError: If shape of starting matrix is different from final matrix
    """
    size = int(pop_matrix.shape[0]*lambda_leak)
    susceptible = pop_matrix[pop_matrix[:, 1] == states_dict['susceptible']][:, 0]

    exposed = np.random.choice(susceptible, size=size, replace=False)

    if len(exposed) == 0:
        return pop_matrix

    new_matrix = expose_population(pop_matrix, exposed, day)

    if new_matrix.shape != pop_matrix.shape:
        raise ValueError("Input and output matrix shapes are different")

    return new_matrix

In [29]:
def update_population(pop_matrix):
    """
    Receives the population matrix and progress the infections
    for all people. The state duration is decremented
    and for those whom it reaches zero, they transition to the next state
    Args:
        pop_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.
    Returns:
        new_matrix (np.array): The population matrix with the updated status.
    Raises:
        ValueError: If shape of starting matrix is different from final matrix
    """
    matrix_keep = pop_matrix[np.isin(pop_matrix[:, 1],
                                     [states_dict['susceptible'],
                                     states_dict['removed']]
                                     )]
    matrix_change = pop_matrix[~np.isin(pop_matrix[:, 1],
                                        [states_dict['susceptible'],
                                        states_dict['removed']]
                                        )]

    matrix_change[:, 3] = matrix_change[:, 3].astype(int) - 1
    matrix_no_change = matrix_change[matrix_change[:, 3].astype(int) > 0]
    matrix_change = matrix_change[matrix_change[:, 3].astype(int) == 0]

    if matrix_change.shape[0] > 0:
        matrix_change = np.apply_along_axis(change_state, 1, matrix_change)

    new_matrix = np.concatenate((matrix_keep, matrix_change, matrix_no_change))

    if new_matrix.shape != pop_matrix.shape:
        raise ValueError("Input and output matrix shapes are different")
    return new_matrix

In [30]:
def spread_infection(pop_matrix, adj_list, restrictions, day, rng, contacts_infected=None):
    """
    Receives the population matrix, the restrictions dictionary and the
    current day. The disease spreads throught the relations in the graph:
    each infected person has a chance to infect a susceptible contact with
    it has an edge in the graph, conditioned to the current restrictions,
    the p_r of each relation.
    Args:
        pop_matrix (np.array): 2D Array  arrays of id, state, day of infection
            and current state duration of population.
        restrictions (dictionary): a dictionary with a value between
        zero and one for each type of relation
        day: the current day of simulation
    Returns:
        new_matrix (np.array): The population matrix with the newly
                                exposed people.
    Raises:
        ValueError: If shape of starting matrix is different from final matrix
    """
    
    def infect_neighbors(neighbors, p_r, restrictions):
        infected = []
        for rel_type, contacts in neighbors.items():
            for c in contacts:
                if rng.random() < p_r[rel_type] * (1-restrictions[rel_type]):
                    infected.append(c)
        
        return infected
    
        exposed = [[item for item, chance in zip(v, rng.random(size=len(v)))
                                     if chance < p_r[k] * (1 - restrictions[k])]
                                     for k,v in neighbors.items()]
        
        return list(chain(*exposed))

    mask = pop_matrix[:, 1] == states_dict['infected']
    currently_infected = pop_matrix[mask][:, 0]

    if currently_infected.shape[0] == 0:
        if contacts_infected is not None:
            return pop_matrix, contacts_infected
        else:
            return pop_matrix
       
    exposed = list(map(lambda x: infect_neighbors(adj_list[x], p_r, restrictions),
                                        currently_infected))
    
    exposed = list(chain(*exposed))

    exposed = np.unique(exposed)
    exposed = exposed.astype(int)
    
        
    mask = np.isin(pop_matrix[:, 0], exposed)
    susceptible = np.isin(pop_matrix[np.array(mask)][:, 1],
                          states_dict['susceptible'])
    exposed = pop_matrix[np.array(mask)][:, 0][susceptible]

    if len(exposed) == 0:
        if contacts_infected is not None:
            return pop_matrix, contacts_infected
        else:
            return pop_matrix
        

    new_matrix = expose_population(pop_matrix, exposed, day)

    if new_matrix.shape != pop_matrix.shape:
        raise ValueError("Input and output matrix shapes are different")


    if contacts_infected is not None:
        return new_matrix, contacts_infected
    else:
        return new_matrix

In [31]:
from tqdm import tqdm

In [33]:
def main(gpickle_path, p_r, policy='Unrestricted',
         days=350, step_size=7,
         disable_tqdm=False, seed=None):
    """
    Receives the policy to be used during the simulation and for how many days
    the simulation should run for. The policy should be a Key in the policies
    dict inside policies.py.
    One full step consists of:
        Spreading the infection
        Exposing through leakage
        Updating the disease evolution of the population
    Args:
        pop_matrix (string): The name of a policy that exists is policies.policies.
        days (int): For how long should the policy run.
    Returns: data (np.array): An array of arrays containing the status of 
    the population at each time step.
    
    """
    
    rng = default_rng(seed)
    pop_matrix, adj_list = init_infection(gpickle_path)
    data = []
    total_steps = int(np.ceil(days/step_size))
    
    print(sys.getsizeof(adj_list))
    
    if isinstance(policy, str):
        policy = total_steps * [policy]
        
    if len(policy) < total_steps:
        raise valueError(f'len of policy should be at least {total_steps}')
    
    for day in tqdm(range(1, days), disable=disable_tqdm):
        # if less than 90% already recovered, break simulation
        if (pop_matrix[pop_matrix[:, 1] == -1].shape[0] > pop_matrix.shape[0]*.9):
            break
            
        if day % step_size == 1:          
            current_step = int(day/step_size)
            restrictions = city_restrictions[policy[current_step]]

        pop_matrix = spread_infection(pop_matrix, adj_list, restrictions, day, rng, p_r)
        pop_matrix = lambda_leak_expose(pop_matrix, day)
        pop_matrix = update_population(pop_matrix)

        data.append(pop_matrix[:, 0:2])

    return data, pop_matrix

In [34]:
prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

g_pickle = '../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle'

In [35]:
%%timeit
policy = "Light Quarantine"
data, pop_matrix = main(g_pickle, p_r, policy=policy)

  3%|██▊                                                                             | 12/349 [00:00<00:02, 112.43it/s]

2621544


 34%|███████████████████████████▎                                                    | 119/349 [00:02<00:05, 40.77it/s]


KeyboardInterrupt: 

In [15]:
import pandas as pd

In [16]:
dfs = [pd.DataFrame(d, columns=['node_id', 'state']) for d in data]
states_counts = [df['state'].value_counts() for df in dfs]
ts = pd.DataFrame(states_counts).reset_index(drop=True).fillna(0)
ts = ts.rename(columns={-1: 'removed', 
                         0: 'susceptible',
                         1: 'exposed',
                         2: 'infected',
                         3: 'hospitalized'})

NameError: name 'data' is not defined

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import datetime
from disease_states import states_dict

set3 = px.colors.qualitative.Set3

color_map_set3 = {
    'Lockdown':          set3[3],
    'Hard Quarantine':    set3[11],
    'Light Quarantine':   set3[1],
    'Social Distancing':  'rgb(204, 245, 175)',
    'Unrestricted':        set3[6]     
}

In [ ]:
def make_SIR_graph(ts):
    fig = go.Figure()
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(ts))
    
    colors = {
        'removed'      : 'green',
        'susceptible'  : 'blue',
        'exposed'      : 'orange',
        'infected'     : 'red',
        'hospitalized' : 'purple'
    }
    
    total = ts.iloc[0].sum()
    
    name_colors = zip()
    
    for c in ts.columns:
        fig.add_trace(go.Scatter(x=x, y=ts[c]/total, name=c, line_color=colors[c]))
    
    fig.update_layout(hovermode='x')
    fig.show()
    
    return None

def make_beds_graph(data, actions, step_size, title, color_map):
    fig = go.Figure()
   
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(data)+2)

    fig.add_trace(go.Scatter(x=x, y=data['hospitalized']/55e3, name='hospitalized', line_color = 'royalblue',
                            line=dict(width=3)))
    fig.add_trace(go.Scatter(x=x, y=data['exposed']/55e3, name='exposed', line_color = 'firebrick',
                            line=dict(width=3)))
    fig.add_trace(go.Scatter(x=x, y=len(data)*[0.0025], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2)))
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=x[step_size*i],
                y0=0,
                x1=x[step_size*(i+1)-1],
                y1=1,
                fillcolor=a,
                opacity=0.5,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)] 
    )

    for k,v in color_map.items():
        fig.add_trace(go.Bar(x=[None], y=[None], marker=dict(color=v), name = k))

    fig.update_layout(coloraxis = {'colorscale':'deep'}, xaxis={'showgrid': False},
                      yaxis = {'showgrid': False},
                      showlegend=True, title = title, hovermode="x")

    fig.show()

In [ ]:
actions = 50*[policy]

In [ ]:
make_SIR_graph(ts)
actions_3_colors = list(map(color_map_set3.get,  actions))
_ = make_beds_graph(ts, actions_3_colors, 7, 'Sim', color_map_set3)